# WeRateDogs Twitter Analysis

This project was done as a Udacity project to put together the concepts we learned. The Twitter handle [WeRateDogs](twitter.com/dog_rates). The key thing this channel does that this project was focused on is extracting the ratings, most of which are 12/10 or some other rating over 10.

As a disclaimer, Udacity is absolute garbage and I would not recomend it to anyone as there is no support. 

## Step 1 - Gathering the Data
There are three data sources and their details are listed below:
* `img_predictions`: This is a file that was gather from Udacity using requests package. The contents include tweet_data with AI predictions as to what the dogs are in the images. 
* `post_archive`: This is a file that was provided from Udacity for download to local machine and included 5,000 posts that has been modified to pull out the dog ratings from the text strings of the posts. Udacity intentionally left key metrics out of this document.
* `tweet_data`: This data source pulls from Twitter's API directly using the tweet_id variable from post_archive. 

The code block below imports our python modules as well as our secrete API credentials so that this notebook can utilize a public git repo.

In [67]:
import requests
import pandas as pd
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer
import os.path
import API_Key_Credentials as creds

consumer_key = creds.consumer_key
consumer_secret = creds.consumer_secret
access_token = creds.access_token
access_secret = creds.access_secret

**Gathering** `img_predictions`

In [68]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

In [69]:
response.status_code

200

In [70]:
if response.status_code == 200:
    print("We're in boys")
else:
    print("Houston, we have a problem, Error: ",response.status_code)

We're in boys


In [71]:
with open('image_predictions.tsv', 'wb') as file:
    file.write(response.content)

In [72]:
img_predictions = pd.read_csv('image_predictions.tsv', delimiter='\t')
img_predictions.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
2015,879492040517615616,https://pbs.twimg.com/media/DDSVWMvXsAEgmMK.jpg,1,German_short-haired_pointer,0.479896,True,vizsla,0.124353,True,bath_towel,0.073320,False
2002,876838120628539392,https://pbs.twimg.com/media/DCsnnZsVwAEfkyi.jpg,1,bloodhound,0.575751,True,redbone,0.240970,True,Tibetan_mastiff,0.088935,True
1612,801538201127157760,https://pbs.twimg.com/media/Cx-itFWWIAAZu7l.jpg,1,Pembroke,0.550506,True,Cardigan,0.306612,True,Shetland_sheepdog,0.054230,True
1210,742465774154047488,https://pbs.twimg.com/media/Ck3EribXEAAPhZn.jpg,1,web_site,0.997154,False,comic_book,0.000439,False,desktop_computer,0.000268,False
1724,819952236453363712,https://pbs.twimg.com/media/C2EONHNWQAUWxkP.jpg,1,American_Staffordshire_terrier,0.925505,True,Staffordshire_bullterrier,0.036221,True,Italian_greyhound,0.020412,True


**Gathering** `post_archive`

In [73]:
post_archive = pd.read_csv('twitter-archive-enhanced.csv')
post_archive.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1453,695629776980148225,NaN,NaN,2016-02-05 15:27:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Calvin. He's proof that degrees mean abso...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/695629776...,8,10,Calvin,None,None,None,None
1218,714957620017307648,NaN,NaN,2016-03-29 23:29:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Curtis. He's an Albino Haberdasher. Te...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/714957620...,10,10,Curtis,None,None,None,None
1258,710283270106132480,NaN,NaN,2016-03-17 01:55:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Gunner. He's a Figamus Newton. King of...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/710283270...,11,10,Gunner,None,None,None,None
2337,666268910803644416,NaN,NaN,2015-11-16 14:57:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Very concerned about fellow dog trapped in com...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666268910...,10,10,None,None,None,None,None
1467,694329668942569472,NaN,NaN,2016-02-02 01:21:07 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Grady. He's very hungry. Too bad no one c...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/694329668...,9,10,Grady,None,None,pupper,None


**Gathering** `tweet_data`

In [74]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

tweet_ids = post_archive.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()


# Save each tweet's returned JSON as a new line in a .txt file
if os.path.exists('tweet_json.txt'):
    print('Twitter Info Already Captured')
else:
    with open('tweet_json.txt', 'w') as outfile:
        # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
        for tweet_id in tweet_ids:
            count += 1
            print(str(count) + ": " + str(tweet_id))
            try:
                tweet = api.get_status(tweet_id, tweet_mode='extended')
                print("Success")
                json.dump(tweet._json, outfile)
                outfile.write('\n')
            except tweepy.TweepError as e:
                print("Fail")
                fails_dict[tweet_id] = e
                pass
end = timer()


Twitter Info Already Captured


This codeblock below took a while to figure out but was my solution to storing the raw JSON from a txt file to python object. It stores each JSON as a dictionary inside the `tweet_data` list. Note that `for line in f:` is a special sytax that parses each line of the txt file.

In [75]:
with open('tweet_json.txt') as f:
    tweet_data = []
    for line in f:
        tweet_data.append(json.loads(line))
    

In [76]:
type(tweet_data)

list

In [77]:
type(tweet_data[1])

dict

Example of the raw API with all it's attributes before selectively choosing which ones to add to our  master dataFrame

In [78]:
tweet_data[4]

{'created_at': 'Sat Jul 29 16:00:24 +0000 2017',
 'id': 891327558926688256,
 'id_str': '891327558926688256',
 'full_text': 'This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f',
 'truncated': False,
 'display_text_range': [0, 138],
 'entities': {'hashtags': [{'text': 'BarkWeek', 'indices': [129, 138]}],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 891327551943041024,
    'id_str': '891327551943041024',
    'indices': [139, 162],
    'media_url': 'http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg',
    'url': 'https://t.co/AtUZn91f7f',
    'display_url': 'pic.twitter.com/AtUZn91f7f',
    'expanded_url': 'https://twitter.com/dog_rates/status/891327558926688256/photo/1',
    'type': 'photo',
    'sizes': {'medium': {'w': 720, 'h': 540, 'resize': 'fit'},
     'large': {'w': 720, 'h':

In [79]:
tweet_data[4]['retweet_count']

8837

## Step 2 - Creating Master dataFrame
This process is all about merging our three datasets then going through the process of accessing and cleaning.

This codeblock transforms our list of dictionaries to a dataFrame. Some did not contain media which would throw an error. Because in this instance we are only interested in posts with photos using a simple pass funtion solved our issue appropiately.

In [80]:
tweets = pd.DataFrame()

for i in range(1,len(tweet_data)):
    try:
        tweets = tweets.append({'tweet_id': tweet_data[i]['id'],
                                'retweets': tweet_data[i]['retweet_count'],
                                'favorite_counts': tweet_data[i]['favorite_count'],
                                'created_at': tweet_data[i]['created_at'],
                                'full_text':tweet_data[i]['full_text'].split('https://')[0],
                                'post_link':tweet_data[i]['full_text'].split('https://')[1],
                                'picture':tweet_data[i]['entities']['media'][0]['media_url'],
                                'retweeted':tweet_data[i]['retweeted']
                               }, ignore_index=True)
    except:
        pass

In [81]:
tweets.columns.tolist()

['created_at',
 'favorite_counts',
 'full_text',
 'picture',
 'post_link',
 'retweeted',
 'retweets',
 'tweet_id']

### Question:
Is it possible to merge three dataFrames at the same time in a pythonic fashion?

In [82]:
master_df = pd.merge(tweets, img_predictions, on='tweet_id')
master_df

,created_at,favorite_counts,full_text,picture,post_link,retweeted,retweets,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,Mon Jul 31 00:18:03 +0000 2017,24183.0,This is Archie. He is a rare Norwegian Pouncin...,http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,t.co/wUnZnhtVJB,0.0,3925.0,8.918152e+17,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
1,Sun Jul 30 15:58:51 +0000 2017,40629.0,This is Darla. She commenced a snooze mid meal...,http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,t.co/tD36da7qLQ,0.0,8161.0,8.916896e+17,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2,Sat Jul 29 16:00:24 +0000 2017,38862.0,This is Franklin. He would like you to stop ca...,http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg,t.co/AtUZn91f7f,0.0,8837.0,8.913276e+17,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
3,Sat Jul 29 00:08:17 +0000 2017,19532.0,Here we have a majestic great white breaching ...,http://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,t.co/kQ04fDDRmh,0.0,2940.0,8.910880e+17,https://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,1,Chesapeake_Bay_retriever,0.425595,True,Irish_terrier,0.116317,True,Indian_elephant,0.076902,False
4,Fri Jul 28 00:22:40 +0000 2017,62928.0,When you watch your owner call another dog a g...,http://pbs.twimg.com/media/DFyBag_UQAAhhBC.jpg,t.co/v0nONBcwxq,0.0,17831.0,8.907292e+17,https://pbs.twimg.com/media/DFyBahAVwAAhUTd.jpg,2,Pomeranian,0.566142,True,Eskimo_dog,0.178406,True,Pembroke,0.076507,True
5,Thu Jul 27 16:25:51 +0000 2017,26866.0,This is Zoey. She doesn't want to be one of th...,http://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,t.co/9TwLuAGH0b,0.0,4041.0,8.906092e+17,https://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,1,Irish_terrier,0.487574,True,Irish_setter,0.193054,True,Chesapeake_Bay_retriever,0.118184,True
6,Wed Jul 26 00:31:25 +0000 2017,29605.0,This is Koda. He is a South Australian decksha...,http://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,t.co/dVPW0B0Mme,0.0,6925.0,8.900066e+17,https://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,1,Samoyed,0.957979,True,Pomeranian,0.013884,True,chow,0.008167,True
7,Tue Jul 25 00:10:02 +0000 2017,26144.0,This is Ted. He does his best. Sometimes that'...,http://pbs.twimg.com/media/DFihzFfXsAYGDPR.jpg,t.co/f8dEDcrKSR,0.0,4251.0,8.896388e+17,https://pbs.twimg.com/media/DFihzFfXsAYGDPR.jpg,1,French_bulldog,0.991650,True,boxer,0.002129,True,Staffordshire_bullterrier,0.001498,True
8,Mon Jul 24 00:19:32 +0000 2017,24343.0,This is Oliver. You're witnessing one of his m...,http://pbs.twimg.com/ext_tw_video_thumb/889278...,t.co/WpHvrQedPb,0.0,5063.0,8.892788e+17,https://pbs.twimg.com/ext_tw_video_thumb/88927...,1,whippet,0.626152,True,borzoi,0.194742,True,Saluki,0.027351,True
9,Sun Jul 23 00:22:39 +0000 2017,28106.0,This is Jim. He found a fren. Taught him how t...,http://pbs.twimg.com/media/DFYRgsOUQAARGhO.jpg,t.co/chxruIOUJN,0.0,4246.0,8.889172e+17,https://pbs.twimg.com/media/DFYRgsOUQAARGhO.jpg,1,golden_retriever,0.714719,True,Tibetan_mastiff,0.120184,True,Labrador_retriever,0.105506,True


In [83]:
master_df = pd.merge(master_df, post_archive, on='tweet_id')
master_df.sample(20)

,created_at,favorite_counts,full_text,picture,post_link,retweeted,retweets,tweet_id,jpg_url,img_num,...,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
350,Mon Sep 26 23:04:13 +0000 2016,6593.0,Here's a perturbed super floof. 12/10 would sn...,http://pbs.twimg.com/media/CtUMLzRXgAAbZK5.jpg,t.co/VG095mi09Q,0.0,1825.0,7.805435e+17,https://pbs.twimg.com/media/CtUMLzRXgAAbZK5.jpg,1,...,NaN,NaN,https://twitter.com/dog_rates/status/780543529...,12,10,None,None,None,None,None
1037,Wed Dec 09 02:56:22 +0000 2015,1437.0,This is Ava. She doesn't understand flowers. 1...,http://pbs.twimg.com/media/CVwHgblWcAACWOD.jpg,t.co/BxTJAFSIgk,0.0,539.0,6.744223e+17,https://pbs.twimg.com/media/CVwHgblWcAACWOD.jpg,1,...,NaN,NaN,https://twitter.com/dog_rates/status/674422304...,12,10,Ava,None,None,None,None
481,Wed Jul 06 15:54:42 +0000 2016,13634.0,This is Meyer. He has to hold somebody's hand ...,http://pbs.twimg.com/media/CmsXg9AWgAAs6Ui.jpg,t.co/WS6BoApYyL,0.0,5242.0,7.507196e+17,https://pbs.twimg.com/media/CmsXg9AWgAAs6Ui.jpg,1,...,NaN,NaN,https://twitter.com/dog_rates/status/750719632...,12,10,Meyer,None,None,None,None
240,Thu Dec 15 17:23:04 +0000 2016,7274.0,"I call this one ""A Blep by the Sea"" 12/10",http://pbs.twimg.com/media/Czu9RiwVEAA_Okk.jpg,t.co/EMdnCugNbo,0.0,1553.0,8.094487e+17,https://pbs.twimg.com/media/Czu9RiwVEAA_Okk.jpg,1,...,NaN,NaN,https://twitter.com/dog_rates/status/809448704...,12,10,None,None,None,None,None
418,Sun Aug 07 15:56:28 +0000 2016,4994.0,This is Scout. He specializes in mid-air freez...,http://pbs.twimg.com/media/CpRKzZKWAAABGh7.jpg,t.co/sAHmwRtfSq,0.0,1173.0,7.623165e+17,https://pbs.twimg.com/media/CpRKzZKWAAABGh7.jpg,1,...,NaN,NaN,https://twitter.com/dog_rates/status/762316489...,11,10,Scout,None,None,None,None
90,Sun Apr 30 00:02:42 +0000 2017,21322.0,This is Sophie. She just arrived. Used paworit...,http://pbs.twimg.com/media/C-nnZBdXkAAB-wg.jpg,t.co/8jOC4zhNxy,0.0,4789.0,8.584716e+17,https://pbs.twimg.com/media/C-nnZBdXkAAB-wg.jpg,1,...,NaN,NaN,https://twitter.com/dog_rates/status/858471635...,13,10,Sophie,None,None,None,None
1017,Thu Dec 10 03:59:15 +0000 2015,3032.0,This is Tedders. He broke his leg saving babie...,http://pbs.twimg.com/media/CV1ffl3XAAAiFyr.jpg,t.co/cMI2AcLm4B,0.0,868.0,6.748005e+17,https://pbs.twimg.com/media/CV1ffl3XAAAiFyr.jpg,1,...,NaN,NaN,https://twitter.com/dog_rates/status/674800520...,11,10,Tedders,None,None,None,None
650,Sat Mar 19 01:54:56 +0000 2016,3063.0,This is Chuckles. He had a balloon but he acci...,http://pbs.twimg.com/media/Cd4CBQFW8AAY3ND.jpg,t.co/68iNM7B5gW,0.0,627.0,7.110080e+17,https://pbs.twimg.com/media/Cd4CBQFW8AAY3ND.jpg,1,...,NaN,NaN,https://twitter.com/dog_rates/status/711008018...,11,10,Chuckles,None,None,pupper,None
1262,Sat Nov 21 17:05:31 +0000 2015,660.0,This is Alfie. He's that one hypocritical gym ...,http://pbs.twimg.com/media/CUWdPsqWcAERQVv.jpg,t.co/GCJzm3YsfX,0.0,236.0,6.681130e+17,https://pbs.twimg.com/media/CUWdPsqWcAERQVv.jpg,1,...,NaN,NaN,https://twitter.com/dog_rates/status/668113020...,6,10,Alfie,None,None,None,None
985,Sun Dec 13 02:51:51 +0000 2015,1653.0,&amp; this is Yoshi. Another world record cont...,http://pbs.twimg.com/media/CWEs1b-WEAEhq82.jpg,t.co/QG708dDNH6,0.0,576.0,6.758707e+17,https://pbs.twimg.com/media/CWEs1b-WEAEhq82.jpg,1,...,NaN,NaN,https://twitter.com/dog_rates/status/675870721...,11,10,None,None,None,None,None


## Step 3 - Cleaning the Master dataFrame

In [84]:
master_df.columns.tolist()

['created_at',
 'favorite_counts',
 'full_text',
 'picture',
 'post_link',
 'retweeted',
 'retweets',
 'tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo']

In [88]:
#Transforms the appropiate fields to datetime format
time_variables = ['created_at','timestamp','retweeted_status_timestamp']
for var in time_variables:
    master_df[var] = pd.to_datetime(master_df[var])

In [87]:
#Checking dTypes
master_df.dtypes

created_at                    datetime64[ns, UTC]
favorite_counts                           float64
full_text                                  object
picture                                    object
post_link                                  object
retweeted                                 float64
retweets                                  float64
tweet_id                                  float64
jpg_url                                    object
img_num                                     int64
p1                                         object
p1_conf                                   float64
p1_dog                                       bool
p2                                         object
p2_conf                                   float64
p2_dog                                       bool
p3                                         object
p3_conf                                   float64
p3_dog                                       bool
in_reply_to_status_id                     float64


In [98]:
#Searching for null values
master_df.isnull().sum()

created_at                       0
favorite_counts                  0
full_text                        0
picture                          0
post_link                        0
retweeted                        0
retweets                         0
tweet_id                         0
jpg_url                          0
img_num                          0
p1                               0
p1_conf                          0
p1_dog                           0
p2                               0
p2_conf                          0
p2_dog                           0
p3                               0
p3_conf                          0
p3_dog                           0
in_reply_to_status_id         1327
in_reply_to_user_id           1327
timestamp                        0
source                           0
text                             0
retweeted_status_id           1297
retweeted_status_user_id      1297
retweeted_status_timestamp    1297
expanded_urls                    0
rating_numerator    

In [101]:
pd.set_option('display.max_columns', 500)
master_df

,created_at,favorite_counts,full_text,picture,post_link,retweeted,retweets,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,2017-07-31 00:18:03+00:00,24183.0,This is Archie. He is a rare Norwegian Pouncin...,http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,t.co/wUnZnhtVJB,0.0,3925.0,8.918152e+17,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,7.825300e-02,True,kelpie,3.137890e-02,True,NaN,NaN,2017-07-31 00:18:03+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
1,2017-07-30 15:58:51+00:00,40629.0,This is Darla. She commenced a snooze mid meal...,http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,t.co/tD36da7qLQ,0.0,8161.0,8.916896e+17,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,1.680860e-01,True,spatula,4.083590e-02,False,NaN,NaN,2017-07-30 15:58:51+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
2,2017-07-29 16:00:24+00:00,38862.0,This is Franklin. He would like you to stop ca...,http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg,t.co/AtUZn91f7f,0.0,8837.0,8.913276e+17,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,2.257700e-01,True,German_short-haired_pointer,1.752190e-01,True,NaN,NaN,2017-07-29 16:00:24+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
3,2017-07-29 00:08:17+00:00,19532.0,Here we have a majestic great white breaching ...,http://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,t.co/kQ04fDDRmh,0.0,2940.0,8.910880e+17,https://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,1,Chesapeake_Bay_retriever,0.425595,True,Irish_terrier,1.163170e-01,True,Indian_elephant,7.690220e-02,False,NaN,NaN,2017-07-29 00:08:17+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
4,2017-07-28 00:22:40+00:00,62928.0,When you watch your owner call another dog a g...,http://pbs.twimg.com/media/DFyBag_UQAAhhBC.jpg,t.co/v0nONBcwxq,0.0,17831.0,8.907292e+17,https://pbs.twimg.com/media/DFyBahAVwAAhUTd.jpg,2,Pomeranian,0.566142,True,Eskimo_dog,1.784060e-01,True,Pembroke,7.650690e-02,True,NaN,NaN,2017-07-28 00:22:40+00:00,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
5,2017-07-27 16:25:51+00:00,26866.0,This is Zoey. She doesn't want to be one of th...,http://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,t.co/9TwLuAGH0b,0.0,4041.0,8.906092e+17,https://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,1,Irish_terrier,0.487574,True,Irish_setter,1.930540e-01,True,Chesapeake_Bay_retriever,1.181840e-01,True,NaN,NaN,2017-07-27 16:25:51+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
6,2017-07-26 00:31:25+00:00,29605.0,This is Koda. He is a South Australian decksha...,http://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,t.co/dVPW0B0Mme,0.0,6925.0,8.900066e+17,https://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,1,Samoyed,0.957979,True,Pomeranian,1.388350e-02,True,chow,8.167480e-03,True,NaN,NaN,2017-07-26 00:31:25+00:00,"<a hre